In [1]:
import pandas as pd
import numpy as np

In [10]:
df = pd.read_csv('/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/20210104.csv')
df.head()

,Unnamed: 0,ticker,open,high,low,close,volume,OPCL,pvCLCL,prevAdjClose,SPpvCLCL,sharesOut,PERMNO,SICCD,PERMCO,prevRawOpen,prevRawClose,prevAdjOpen
0,1,SPY,375.31000,375.45001,364.82001,368.79001,110210810,-0.017525,-0.013614,373.88,-0.014755,886482,84398,6726,46699,371.78000,373.88,371.79
1,2,IWM,197.53999,197.89000,190.94000,193.50000,33664242,-0.020664,-0.013057,196.06,-0.014755,298950,88222,6726,37493,196.62000,196.06,196.62
2,3,EEM,52.62000,52.81000,51.83000,52.00000,48201262,-0.011853,0.006387,51.67,-0.014755,546300,89730,6726,37493,51.99000,51.67,51.99
3,4,TLT,156.67000,158.17999,156.53500,157.53999,13140997,0.005538,-0.001205,157.73,-0.014755,120800,89468,6726,37493,157.46001,157.73,157.46
4,5,USO,33.00000,33.29000,32.24000,32.38000,14340206,-0.018967,-0.019085,33.01,-0.014755,131524,91208,6221,50617,32.73000,33.01,32.73
